# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [6]:
# Write your code below.
import os
from dotenv import load_dotenv

dotenv_path = '../../01_materials/labs/.env'
load_dotenv(dotenv_path=dotenv_path)

True

In [4]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [12]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv('PRICE_DATA')
print("Folder structure pointing to PRICE_DATA in env is : ", PRICE_DATA)
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
print("Total number of parquest files in the {} is {}".format(PRICE_DATA,len(parquet_files)))



Folder structure pointing to PRICE_DATA in env is :  ../../05_src/data/prices/
Total number of parquest files in the ../../05_src/data/prices/ is 11207


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [32]:
# Write your code below.
import warnings
warnings.filterwarnings("ignore")
dd_px = dd.read_parquet(PRICE_DATA).set_index("ticker") # load parquet files into dask data frame
print("Columns in dask dataframe : \n", dd_px.columns) # columns in dataframe
print("\n\n Dask Data frame :\n", dd_px.head(5))

# Feature data set transformation
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    positive_return = lambda x: (x['returns'] > 0)*1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))

print("\n\nDask Feature data frame columns after transformation :\n ",dd_feat.columns)
print("\n\n Dask Feature Data set after transformation: \n\n",dd_feat.head(5))

Columns in dask dataframe : 
 Index(['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object', name='Price')


 Dask Data frame :
 Price                       Date  Adj Close      Close       High        Low  \
ticker                                                                         
A      2000-01-03 00:00:00+00:00  43.463020  51.502148  56.464592  48.193848   
A      2000-01-04 00:00:00+00:00  40.142925  47.567955  49.266811  46.316166   
A      2000-01-05 00:00:00+00:00  37.652859  44.617310  47.567955  43.141991   
A      2000-01-06 00:00:00+00:00  36.219196  42.918453  44.349072  41.577251   
A      2000-01-07 00:00:00+00:00  39.237450  46.494991  47.165592  42.203148   

Price        Open   Volume       sector                       subsector  year  
ticker                                                                         
A       56.330471  4674353  Health Care  Life Sciences Tools & Services  2000  
A 

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [38]:
# Write your code below.

dd_pd = dd_feat.compute() # covert to panda data frame
dd_pd['rolling_mean'] = dd_pd['returns'].rolling(10).mean() # add rolling mean col
dd_pd


Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year,Close_lag_1,returns,positive_return,hi_lo_range,rolling_mean
ticker,,,,,,,,,,,,,,,
HUM,2008-01-02 00:00:00+00:00,69.753014,78.120003,79.139999,76.510002,76.510002,2224700,Health Care,Managed Health Care,2008,NaN,NaN,0,2.629997,NaN
HUM,2008-01-03 00:00:00+00:00,71.985268,80.620003,81.120003,77.809998,78.489998,2426800,Health Care,Managed Health Care,2008,78.120003,0.032002,1,3.310005,NaN
HUM,2008-01-04 00:00:00+00:00,70.958420,79.470001,81.110001,78.949997,80.099998,1619200,Health Care,Managed Health Care,2008,80.620003,-0.014264,0,2.160004,NaN
HUM,2008-01-07 00:00:00+00:00,72.824577,81.559998,81.610001,79.110001,80.110001,1984500,Health Care,Managed Health Care,2008,79.470001,0.026299,1,2.500000,NaN
HUM,2008-01-08 00:00:00+00:00,73.146004,81.919998,83.070000,81.470001,82.040001,2468000,Health Care,Managed Health Care,2008,81.559998,0.004414,1,1.599998,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2008-12-24 00:00:00+00:00,13.271270,17.030001,17.090000,16.730000,16.840000,2079800,Information Technology,Communications Equipment,2008,16.820000,0.012485,1,0.360001,-0.004039
JNPR,2008-12-26 00:00:00+00:00,13.193337,16.930000,17.180000,16.700001,17.160000,1269100,Information Technology,Communications Equipment,2008,17.030001,-0.005872,0,0.480000,-0.000917
JNPR,2008-12-29 00:00:00+00:00,13.349196,17.129999,17.440001,16.740000,17.020000,4528900,Information Technology,Communications Equipment,2008,16.930000,0.011813,1,0.700001,-0.001662


In [36]:
print(f"Type of `dd_feat`: {type(dd_feat)}")
print(f"Type of `dd_pd`: {type(dd_pd)}")

Type of `dd_feat`: <class 'dask.dataframe.core.DataFrame'>
Type of `dd_pd`: <class 'pandas.core.frame.DataFrame'>


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

**Answer**

Both Pandas DataFrames and Dask DataFrames have their strengths and weaknesses, making them suitable for different scenarios. In our scenario we are dealing with relatively small datasets, converting to Pandas is not a bad idea. Pandas library has a ease to use and rich functionalitu that is used in data analysis and visualization. It comes with benefit of in-memory processing which leads to fast execution for our datasets that fit within available memory. 

On the other hand, if we are working with big data and which requires efficient parallel or distributed computing, Dask DataFrames offer an impressive solution. Our choice ultimately depends on the size of our data, the complexity of our tasks, and most important willingness to explore and adopt new tools for optimal performance and scalability.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.